In [1]:
"""
CNN on CIFAR10 dataset

With Augmentation:
    76% Validation Accuracy @Epoch 15
    78% Validation Accuracy @Epoch 20
    80% Validation Accuracy @Epoch 25
    84% Validation Accuracy @Epoch 50
    87% Validation Accuracy @Epoch 100

    38ms/step on an RTX 2070
    64 minutes and 20 sec to complete 100 Epochs
    1000 steps per Epoch

Without Augmentation:
    75% Validation Accuracy @Epoch 15
    78% Validation Accuracy @Epoch 25
    81% Validation Accuracy @Epoch 50
    83% Validation Accuracy @Epoch 100
    
    ~318us/step on an RTX 2070
    26 minutes and 21 seconds to complete 100 Epochs
    50000 steps per Epoch
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import time
import numpy as np
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical, plot_model, Sequence
from keras import backend as K
import os

if K.backend() == 'tensorflow': #prevents out CUDA out of memory
    import tensorflow as tf
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)

stime = time.time()

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

batch_size = 128 
#WARNING: This depth setup (256->512->512) utilizes 90% of the GPU using an RTX 2070  
depth1 = 256 
depth2 = 512
depth3 = 512
hidden_units = 1024
num_classes = 10
epochs = 100
num_predictions = 20
image_size = x_train.shape[1]
input_shape = (image_size, image_size, 3)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(depth1, (3, 3), padding='same', activation = 'relu', input_shape=x_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(depth2, (3, 3), padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(depth2, (3, 3), padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()
plot_model(model, to_file='cnn-mnist.png', show_shapes=True)


opt = keras.optimizers.adam(lr=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

data_augmentation = True
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1, # randomly shift images vertically (fraction of total height)
        fill_mode='nearest',
        horizontal_flip=True  # randomly flip images
        )
    datagen.fit(x_train)

    model.fit_generator(datagen.flow(x_train, y_train,
        batch_size=batch_size),
        epochs=epochs,
        validation_data=(x_test, y_test),
        use_multiprocessing = True,
        workers=12, #number of cores
        steps_per_epoch = 1000)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

etime = time.time()
print('Time taken in min: ', (etime-stime)/60)

Using TensorFlow backend.
W0308 19:02:29.063536 140044213495616 deprecation.py:506] From /home/chico/.virtualenvs/dl4cv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 256)       7168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 256)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 512)       1180160   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 512)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 512)         0         
_________________________________________________________________
con

/home/chico/.virtualenvs/dl4cv/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


1000/1000 [==============================] - 40s 40ms/step - loss: 1.6830 - acc: 0.3808 - val_loss: 1.4285 - val_acc: 0.4819
Epoch 2/100
1000/1000 [==============================] - 37s 37ms/step - loss: 1.3277 - acc: 0.5251 - val_loss: 1.2370 - val_acc: 0.5677
Epoch 3/100
1000/1000 [==============================] - 36s 36ms/step - loss: 1.1253 - acc: 0.6014 - val_loss: 1.0889 - val_acc: 0.6172
Epoch 4/100
1000/1000 [==============================] - 37s 37ms/step - loss: 1.0266 - acc: 0.6416 - val_loss: 0.9853 - val_acc: 0.6589
Epoch 5/100
1000/1000 [==============================] - 38s 38ms/step - loss: 0.9173 - acc: 0.6813 - val_loss: 0.8890 - val_acc: 0.6909
Epoch 6/100
1000/1000 [==============================] - 38s 38ms/step - loss: 0.8460 - acc: 0.7039 - val_loss: 0.8835 - val_acc: 0.6934
Epoch 7/100
1000/1000 [==============================] - 38s 38ms/step - loss: 0.8208 - acc: 0.7140 - val_loss: 0.9020 - val_acc: 0.6971
Epoch 8/100
1000/1000 [==============================

1000/1000 [==============================] - 39s 39ms/step - loss: 0.3008 - acc: 0.8950 - val_loss: 0.4875 - val_acc: 0.8492
Epoch 61/100
1000/1000 [==============================] - 38s 38ms/step - loss: 0.3003 - acc: 0.8964 - val_loss: 0.4693 - val_acc: 0.8568
Epoch 62/100
1000/1000 [==============================] - 38s 38ms/step - loss: 0.2732 - acc: 0.9066 - val_loss: 0.4535 - val_acc: 0.8557
Epoch 63/100
1000/1000 [==============================] - 38s 38ms/step - loss: 0.2816 - acc: 0.9011 - val_loss: 0.4786 - val_acc: 0.8528
Epoch 64/100
1000/1000 [==============================] - 38s 38ms/step - loss: 0.2882 - acc: 0.9006 - val_loss: 0.4810 - val_acc: 0.8506
Epoch 65/100
1000/1000 [==============================] - 38s 38ms/step - loss: 0.2902 - acc: 0.8996 - val_loss: 0.4844 - val_acc: 0.8459
Epoch 66/100
1000/1000 [==============================] - 38s 38ms/step - loss: 0.2886 - acc: 0.8997 - val_loss: 0.4609 - val_acc: 0.8550
Epoch 67/100
1000/1000 [=======================